In [1]:
### THIS SCRIPT COLLECTS DATA FROM SEP and InPhO ### 

In [2]:
# import dependencies
import time
import pymongo
import re
import json
import requests
import pandas as pd

from tqdm import tqdm
from pprint import pprint
from pathlib import Path
from bs4 import BeautifulSoup

#import local libraries
import lib_sepinpho as sep
import lib_fileops as io

In [3]:
##### INIT GLOBAL VARIABLES#####

#init Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

#connect to database
db = client.visualizing_sep

In [8]:
### SCRAPE TOC FROM WINTER 2019 SEP ARCHIVE ###
### Date Completed: 3.30.2020 ###

#set variable options for Winter 2019 download
# toc_collection = db.toc_2019_winter
# sep_edition = 'Winter_2019'
# search_url = 'https://plato.stanford.edu/archives/win2019/contents.html'
# base_url = 'https://plato.stanford.edu/archives/win2019/'
# save_as_html = '../downloads/tocs/toc_2019_winter_all.html'

# print(f'Starting: SEP {sep_edition} TOC Scrape\n')

# #get list of elements to save into toc_collection 
# toc_elements = sep.scrape_sep_toc(sep_edition, search_url, base_url, save_as_html)

# #update MonogDB collection
# toc_collection.insert_many(toc_elements)

# print(f'Success! Completed SEP {sep_edition} TOC Scrape\n')

Starting: SEP Winter_2019 TOC Scrape

Success! Completed SEP Winter_2019 TOC Scrape



In [ ]:
### DOWNLOAD WINTER 2019 SEP ARCHIVE, AND STORE THEM LOCALLY###
# Winter 2019 Archive: Completed 1/30/2020. This took 7 hours, so don't do it again!
# Updated the code on 4.10.2020 to make the download process cleaner, but the files were NOT re-downloaded

#set variable options for Winter 2019 download
# toc_collection = db.toc_2019_winter
# toc_pages = list(toc_collection.find().sort('link_url'))
# save_as_directory = '..downloads/articles/sep_2019_all/'

#download SEP pages from archive
#sep.download_sep_pages(toc_pages,save_as_directory)

In [9]:
### SCRAPE WHATS NEW TOC FROM SPRING 2020 SEP ARCHIVE ###
### Downloaded Spring 2020 TOC updates: completed 4.10.2020 ###

#set variable options for Spring 2020 download
# toc_collection = db.toc_2020_spring_new
# sep_edition = 'Spring_2020'
# search_url = 'https://plato.stanford.edu/archives/spr2020/new.html'
# base_url = 'https://plato.stanford.edu/archives/spr2020/'
# save_as_html = '../downloads/tocs/toc_2020_spring_new.html'

# print(f'Starting: SEP {sep_edition} TOC Scrape\n')

# toc_elements = sep.scrape_sep_toc(sep_edition, search_url, base_url, save_as_html)
# toc_collection.insert_many(toc_elements)

# print(f'Success! Completed SEP {sep_edition} TOC Scrape\n')


Starting: SEP Spring_2020 TOC Scrape

Success! Completed SEP Spring_2020 TOC Scrape



In [4]:
### DOWNLOAD SPRING 2020 WHATS NEW FILES, AND STORE THEM LOCALLY###
#Spring 2020 Archive: Completed 4.10.2020#

#set variable options for Spring 2020 download
# toc_collection = db.toc_2020_spring_new
# toc_pages = list(toc_collection.find().sort('link_url'))
# save_as_directory = '../downloads/articles/sep_2020_spring/'

# #download SEP pages from archive
# sep.download_sep_pages(toc_pages,save_as_directory)

Processing: 100%|██████████| 67/67 [15:53<00:00, 14.23s/it]


In [ ]:
### PARSING Winter 2019 and Spring 2020 files ###

### After downloading the Winter 2019 archive and the Spring 2020 updates, I moved the updates into the same folder, and then ran the parsing script on all of the files at once. This replaced the prior verion of the parsed data objects in MongoDB. 
###

#set variable options for parsing
# collection_to_update = db.sep_entries
# file_path='../downloads/articles/sep_2019_all'
# file_type='*.html'
# sep_files = io.get_local_files(file_path,file_type)
# for sep_file in tqdm(sep_files):
#     try:
#         sep_object = sep.parse_sep_file(sep_file)
#         collection_to_update.insert_one(sep_object)
#     except:
#         print(f'Failed at: @{sep_file.name}')

In [10]:
### SCRAPE WHATS NEW TOC FROM SUMMER 2020 SEP ARCHIVE ###
### Downloaded Summer 2020 TOC updates: completed 7.13.2020 ###

#set variable options for Summer 2020 download
# toc_collection = db.toc_2020_summer_new
# sep_edition = 'Summer_2020'
# search_url = 'https://plato.stanford.edu/archives/sum2020/new.html'
# base_url = 'https://plato.stanford.edu/archives/sum2020/'
# save_as_html = '../downloads/tocs/toc_2020_summer_new.html'

# print(f'Starting: SEP {sep_edition} TOC Scrape\n')

# toc_elements = sep.scrape_sep_toc(sep_edition, search_url, base_url, save_as_html)
# toc_collection.insert_many(toc_elements)

# print(f'Success! Completed SEP {sep_edition} TOC Scrape\n')


Starting: SEP Summer_2020 TOC Scrape

Success! Completed SEP Summer_2020 TOC Scrape



In [4]:
### DOWNLOAD SUMMER 2020 WHATS NEW FILES, AND STORE THEM LOCALLY###
#Summer 2020 Archive: Completed 7.13.2020#

#set variable options for Summer 2020 download
# toc_collection = db.toc_2020_summer_new
# toc_pages = list(toc_collection.find().sort('link_url'))
# save_as_directory = '../downloads/articles/sep_2020_summer/'

# #download SEP pages from archive
# sep.download_sep_pages(toc_pages,save_as_directory)

Processing: 0it [00:00, ?it/s]


In [ ]:
#Update collection for Summer 2020 files after new files downloaded
### PARSING  Summer 2020 files ###

# collection_to_update = db.sep_entries
# file_path='../downloads/articles/sep_2020_summer'
# file_type='*.html'
# sep_files = io.get_local_files(file_path,file_type)
# for sep_file in tqdm(sep_files):
#     sep_object = sep.parse_sep_file(sep_file)
#     update_string = sep.update_main_document_data(sep_object)
#     find = {"page_url": sep_object['page_url']}
#     collection_to_update.update_one(find, {'$set': update_string}, upsert=True)

In [7]:
### SCRAPE WHATS NEW TOC FROM FALL 2020 SEP ARCHIVE ###
### Downloaded Fall 2020 TOC updates: completed 10.1.2020 ###

#set variable options for Fall 2020 download
# toc_collection = db.toc_2020_fall_new
# sep_edition = 'Fall_2020'
# search_url = 'https://plato.stanford.edu/archives/fall2020/new.html'
# base_url = 'https://plato.stanford.edu/archives/fall2020/'
# save_as_html = '../downloads/tocs/toc_2020_fall_new.html'

# print(f'Starting: SEP {sep_edition} TOC Scrape\n')

# toc_elements = sep.scrape_sep_toc(sep_edition, search_url, base_url, save_as_html)
# toc_collection.insert_many(toc_elements)

# print(f'Success! Completed SEP {sep_edition} TOC Scrape\n')

Starting: SEP Fall_2020 TOC Scrape

Success! Completed SEP Fall_2020 TOC Scrape



In [8]:
### DOWNLOAD FALL 2020 WHATS NEW FILES, AND STORE THEM LOCALLY###
#Fall 2020 Archive: Completed 10.1.2020#

#set variable options for Summer 2020 download
# toc_collection = db.toc_2020_fall_new
# toc_pages = list(toc_collection.find().sort('link_url'))
# save_as_directory = '../downloads/articles/sep_2020_fall/'

# #download SEP pages from archive
# sep.download_sep_pages(toc_pages,save_as_directory)

Processing: 100%|██████████| 87/87 [19:57&lt;00:00, 13.76s/it]


In [ ]:
#Update collection for Fall 2020 files after new files downloaded
#Update completed 10.1.2020#

# collection_to_update = db.sep_entries 
# file_path='../downloads/articles/sep_2020_fall'
# file_type='*.html'
# sep_files = io.get_local_files(file_path,file_type)
# for sep_file in tqdm(sep_files):
#     sep_object = sep.parse_sep_file(sep_file)
#     update_string = sep.update_main_document_data(sep_object)
#     find = {"page_url": sep_object['page_url']}
#     collection_to_update.update_one(find, {'$set': update_string}, upsert=True)

In [ ]:
#export copyright data into Pandas to retrieve author info from 
## Fall 2020 completed 10.1.2020#

# collection_to_update = db.sep_entries

# copyright_entries = pd.DataFrame(list(collection_to_update.find(filter={'author': {'$exists': False}}, 
#                                                            projection=['page_url','copyright'])))

# #strip HTML from copyright data, and re-write the copyright column data
# copyright_entries['copyright'] = copyright_entries.apply(lambda row:sep.get_author_string(row['copyright']), axis=1)
# #export CSV for cleaning
# copyright_entries.to_csv('copyright_fall.csv')

In [ ]:
#update author data for SEP
## Fall 2020 completed 10.1.2020#

# collection_to_update = db.sep_entries

# #import cleaned data
# authors = pd.read_csv('copyright_fall.csv')

# for index,row in tqdm(authors.iterrows()):
#     page_url = row['page_url']
#     authors = row['copyright']
#     sep.add_author(page_url, authors, collection_to_update)

In [17]:
# #create dupe collection to test updates before their made on main collection
# docs = db.sep_entries.find({})
# # db.sep_entries_dupe.drop_collection()
# db.sep_entries_dupe.insert_many(docs)

In [5]:
### SCRAPE WHATS NEW TOC FROM Winter 2020 SEP ARCHIVE ###
### Downloaded Winter 2020 TOC updates: completed 12.24.2020 ###

#set variable options for Winter 2020 download
# toc_collection = db.toc_2020_winter_new
# sep_edition = 'Winter_2020'
# search_url = 'https://plato.stanford.edu/archives/win2020/new.html'
# base_url = 'https://plato.stanford.edu/archives/win2020/'
# save_as_html = '../downloads/tocs/toc_2020_win_new.html'

# print(f'Starting: SEP {sep_edition} TOC Scrape\n')

# toc_elements = sep.scrape_sep_toc(sep_edition, search_url, base_url, save_as_html)
# toc_collection.insert_many(toc_elements)

# print(f'Success! Completed SEP {sep_edition} TOC Scrape\n')

Starting: SEP Winter_2020 TOC Scrape

Success! Completed SEP Winter_2020 TOC Scrape



In [6]:
### DOWNLOAD Winter 2020 WHATS NEW FILES, AND STORE THEM LOCALLY###
#Winter 2020 Archive: Completed 12.24.2020#

#set variable options for Summer 2020 download
# toc_collection = db.toc_2020_winter_new
# toc_pages = list(toc_collection.find().sort('link_url'))
# save_as_directory = '../downloads/articles/sep_2020_winter/'

# #download SEP pages from archive
# sep.download_sep_pages(toc_pages,save_as_directory)

Processing: 100%|██████████| 84/84 [20:48<00:00, 14.86s/it]


In [19]:
#Update collection for Winter 2020 files after new files downloaded
#Update completed 12.24.2020#

# collection_to_update = db.sep_entries
# file_path='../downloads/articles/sep_2020_winter'
# file_type='*.html'
# sep_files = io.get_local_files(file_path,file_type)
# for sep_file in tqdm(sep_files):
#     sep_object = sep.parse_sep_file(sep_file)
#     update_string = sep.update_main_document_data(sep_object)
#     find = {"page_url": sep_object['page_url']}
#     collection_to_update.update_one(find, {'$set': update_string}, upsert=True)

 20%|██        | 17/84 [00:16<00:56,  1.19it/s]Error: No InPhO entry
http://inpho.cogs.indiana.edu/idea/1410.json
 42%|████▏     | 35/84 [00:36<00:49,  1.02s/it]Error: No InPhO entry
http://inpho.cogs.indiana.edu/thinker/3076.json
 46%|████▋     | 39/84 [00:39<00:37,  1.21it/s]Error: No InPhO entry
http://inpho.cogs.indiana.edu/thinker/3599.json
 52%|█████▏    | 44/84 [00:44<00:35,  1.12it/s]Error: No InPhO entry
http://inpho.cogs.indiana.edu/thinker/3584.json
100%|██████████| 84/84 [01:27<00:00,  1.04s/it]


In [11]:
#Manual Author and Dataset Updates 
## Winter 2020 completed 10.26.2020#
# There were only two entirely new entries in this set of updates. There were 4 other updates that replaced prior articles. 
# Because of this, I just hand-updated the entires in MongoDB itself

,_id,page_url,copyright
0,5fe77411f50bb5989124a2ff,/entries/culture/,"<div id=""article-copyright""> <p> <a href=""../...."
1,5fe77424f50bb5989124a316,/entries/freedom-ancient/,"<div id=""article-copyright""> <p> <a href=""../...."


In [4]:
### SCRAPE WHATS NEW TOC FROM Spring 2021 SEP ARCHIVE ###
### Downloaded Spring 2021 TOC updates: completed 4.3.2021 ###

#set variable options for Spring 2021 download
# toc_collection = db.toc_2021_spring_new
# sep_edition = 'Spring_2021'
# search_url = 'https://plato.stanford.edu/archives/spr2021/new.html'
# base_url = 'https://plato.stanford.edu/archives/spr2021/'
# save_as_html = '../downloads/tocs/toc_2021_win_new.html'

# print(f'Starting: SEP {sep_edition} TOC Scrape\n')

# toc_elements = sep.scrape_sep_toc(sep_edition, search_url, base_url, save_as_html)
# toc_collection.insert_many(toc_elements)

# print(f'Success! Completed SEP {sep_edition} TOC Scrape\n')

Starting: SEP Spring_2021 TOC Scrape

Success! Completed SEP Spring_2021 TOC Scrape



In [5]:
### DOWNLOAD Spring 2021 WHATS NEW FILES, AND STORE THEM LOCALLY###
#Spring 2021 Archive: Completed 4.2.2021#

# #set variable options for Spring 2021 download
# toc_collection = db.toc_2021_spring_new
# toc_pages = list(toc_collection.find().sort('link_url'))
# save_as_directory = '../downloads/articles/sep_2021_spring/'

# # #download SEP pages from archive
# sep.download_sep_pages(toc_pages,save_as_directory)

Processing: 100%|██████████| 72/72 [18:13<00:00, 15.19s/it]


In [6]:
#create dupe collection to test updates before their made on main collection
docs = db.sep_entries.find({})
# db.sep_entries_dupe.drop_collection()
db.sep_entries_through2020.insert_many(docs)

In [7]:
#Update collection for Spring 2021 files after new files downloaded
#Update completed 4.3.2021#

# collection_to_update = db.sep_entries
# file_path='../downloads/articles/sep_2021_spring'
# file_type='*.html'
# sep_files = io.get_local_files(file_path,file_type)
# for sep_file in tqdm(sep_files):
#     sep_object = sep.parse_sep_file(sep_file)
#     update_string = sep.update_main_document_data(sep_object)
#     find = {"page_url": sep_object['page_url']}
#     collection_to_update.update_one(find, {'$set': update_string}, upsert=True)

 97%|█████████▋| 70/72 [01:13<00:01,  1.09it/s]Error: No InPhO entry
http://inpho.cogs.indiana.edu/thinker/4075.json
100%|██████████| 72/72 [01:15<00:00,  1.05s/it]


In [4]:
#export copyright data into Pandas to retrieve author info  
## Spring 2021 completed 4.3.2021#

# collection_to_update = db.sep_entries

# copyright_entries = pd.DataFrame(list(collection_to_update.find(filter={'author': {'$exists': False}}, 
#                                                            projection=['page_url','copyright'])))

# # #strip HTML from copyright data, and re-write the copyright column data
# copyright_entries['copyright'] = copyright_entries.apply(lambda row:sep.get_author_string(row['copyright']), axis=1)
# #export CSV for cleaning
# copyright_entries.to_csv('copyright_spring2021.csv')

In [10]:
#update author data for SEP
# Spring 2021 completed 4.3.2021#

# collection_to_update = db.sep_entries

# #import cleaned data
# authors = pd.read_csv('copyright_spring2021.csv')

# for index,row in tqdm(authors.iterrows()):
#     page_url = row['page_url']
#     authors = row['copyright']
#     sep.add_author(page_url, authors, collection_to_update)


# There were also two other articles that needed author updates on: 
# https://plato.stanford.edu/archives/spr2021/entries/cognition-animal/ 
# https://plato.stanford.edu/archives/spr2021/entries/conscience-medieval/

SyntaxError: invalid syntax (<ipython-input-10-7ddd50f224dd>, line 15)

In [6]:
### SCRAPE WHATS NEW TOC FROM Summer 2021 SEP ARCHIVE ###
### Downloaded Summer 2021 TOC updates: completed 9.27.2021 ###
### SCRAPE WHATS NEW TOC FROM Fall 2021 SEP ARCHIVE ###
### Downloaded Fall 2021 TOC updates: completed 9.27.2021 ###


#set variable options for Fall 2021 download
# toc_collection = db.toc_2021_fall_new
# sep_edition = 'Fall_2021'
# search_url = 'https://plato.stanford.edu/archives/fall2021/new.html'
# base_url = 'https://plato.stanford.edu/archives/fall2021/'
# save_as_html = '../downloads/tocs/toc_2021_fall_new.html'

# print(f'Starting: SEP {sep_edition} TOC Scrape\n')

# toc_elements = sep.scrape_sep_toc(sep_edition, search_url, base_url, save_as_html)
# toc_collection.insert_many(toc_elements)

# print(f'Success! Completed SEP {sep_edition} TOC Scrape\n')

Starting: SEP Fall_2021 TOC Scrape

Success! Completed SEP Fall_2021 TOC Scrape



In [8]:
### DOWNLOAD Summer 2021 WHATS NEW FILES, AND STORE THEM LOCALLY###
#Summer and Fall 2021 Archive: Completed 9.27.2021#

# #set variable options for Spring 2021 download
# toc_collection = db.toc_2021_fall_new
# toc_pages = list(toc_collection.find().sort('link_url'))
# save_as_directory = '../downloads/articles/sep_2021_fall/'

# # #download SEP pages from archive
# sep.download_sep_pages(toc_pages,save_as_directory)

Processing: 100%|██████████| 86/86 [22:11<00:00, 15.49s/it]


In [20]:
#Update collection for Spring 2021 files after new files downloaded
#Update completed 4.3.2021#
# Updates for Summer and Fall 2021: completed 9.27.2021

# collection_to_update = db.sep_entries
# file_path='../downloads/articles/sep_2021_fall'
# file_type='*.html'
# sep_files = io.get_local_files(file_path,file_type)
# for sep_file in tqdm(sep_files):
#     sep_object = sep.parse_sep_file(sep_file)
#     update_string = sep.update_main_document_data(sep_object)
#     find = {"page_url": sep_object['page_url']}
#     collection_to_update.update_one(find, {'$set': update_string}, upsert=True)

 16%|█▋        | 14/86 [00:14<01:12,  1.01s/it]Error: No InPhO entry
http://inpho.cogs.indiana.edu/idea/1490.json
 20%|█▉        | 17/86 [00:19<01:29,  1.29s/it]Error: No InPhO entry
http://inpho.cogs.indiana.edu/thinker/2903.json
 52%|█████▏    | 45/86 [00:47<00:34,  1.19it/s]Error: No InPhO entry
http://inpho.cogs.indiana.edu/idea/1181.json
 71%|███████   | 61/86 [01:05<00:23,  1.06it/s]Error: No InPhO entry
http://inpho.cogs.indiana.edu/idea/5458.json
 93%|█████████▎| 80/86 [01:26<00:05,  1.01it/s]Error: No InPhO entry
http://inpho.cogs.indiana.edu/idea/1717.json
100%|██████████| 86/86 [01:36<00:00,  1.12s/it]


In [21]:
#export copyright data into Pandas to retrieve author info  
## Summer and Fal 2021 completed 9.27.2021#

# collection_to_update = db.sep_entries

# copyright_entries = pd.DataFrame(list(collection_to_update.find(filter={'author': {'$exists': False}}, 
#                                                            projection=['page_url','copyright'])))

# # #strip HTML from copyright data, and re-write the copyright column data
# copyright_entries['copyright'] = copyright_entries.apply(lambda row:sep.get_author_string(row['copyright']), axis=1)
# #export CSV for cleaning
# copyright_entries.to_csv('coopyright_summer201_test1_test.csv')

In [22]:
#update author data for SEP
# Spring 2021 completed 4.3.2021#\
# Summer and Fall 2021 completed 9.27.2021

# collection_to_update = db.sep_entries

# #import cleaned data
# authors = pd.read_csv('copyright_summer2021.csv')

# for index,row in tqdm(authors.iterrows()):
#     page_url = row['page_url']
#     authors = row['copyright']
#     sep.add_author(page_url, authors, collection_to_update)




22it [00:00, 331.74it/s]acknowledged: /entries/abu-bakr-al-razi/
 True
acknowledged: /entries/cudworth/
 True
acknowledged: /entries/fuller-margaret/
 True
acknowledged: /entries/hegel-social-political/
 True
acknowledged: /entries/hutcheson/
 True
acknowledged: /entries/infinity/
 True
acknowledged: /entries/legal-probabilism/
 True
acknowledged: /entries/neoliberalism/
 True
acknowledged: /entries/sin-christian/
 True
acknowledged: /entries/understanding/
 True
acknowledged: /entries/africana-contemporary/
 True
acknowledged: /entries/argument/
 True
acknowledged: /entries/causation-regularity/
 True
acknowledged: /entries/critical-phil-race/
 True
acknowledged: /entries/hist-westphilmusic-since-1800/
 True
acknowledged: /entries/hist-westphilmusic-to-1800/
 True
acknowledged: /entries/ibn-rushd/
 True
acknowledged: /entries/legal-interpretation/
 True
acknowledged: /entries/moral-phenomenology/
 True
acknowledged: /entries/public-goods/
 True
acknowledged: /entries/self-defense/
 Tr

In [ ]:
### Winter 2021 ### 

In [6]:

### SCRAPE WHATS NEW TOC FROM Winter 2021 SEP ARCHIVE ###
### Downloaded Fall 2021 TOC updates: completed 1.3.2022 ###


#set variable options for Fall 2021 download
# toc_collection = db.toc_2021_winter_new
# sep_edition = 'Winter_2021'
# search_url = 'https://plato.stanford.edu/archives/win2021/new.html'
# base_url = 'https://plato.stanford.edu/archives/win2021/'
# save_as_html = '../downloads/tocs/toc_2021_win_new.html'

# print(f'Starting: SEP {sep_edition} TOC Scrape\n')

# toc_elements = sep.scrape_sep_toc(sep_edition, search_url, base_url, save_as_html)
# toc_collection.insert_many(toc_elements)

# print(f'Success! Completed SEP {sep_edition} TOC Scrape\n')

Starting: SEP Winter_2021 TOC Scrape

Success! Completed SEP Winter_2021 TOC Scrape



In [7]:
### DOWNLOAD Winter 2021 WHATS NEW FILES, AND STORE THEM LOCALLY###
#Winter 2021 Archive: Completed 1.3.2022#

# #set variable options for Spring 2021 download
# toc_collection = db.toc_2021_winter_new
# toc_pages = list(toc_collection.find().sort('link_url'))
# save_as_directory = '../downloads/articles/sep_2021_winter/'

# # # #download SEP pages from archive
# sep.download_sep_pages(toc_pages,save_as_directory)

Processing: 100%|██████████| 95/95 [24:24<00:00, 15.42s/it]


In [14]:
#create dupe collection to test updates before their made on main collection
# Jan 1.3.2022

# docs = db.sep_entries.find({})
# db.sep_entries_orig.insert_many(docs)   

In [16]:
#Update collection for Winter 2021 files after new files downloaded
# Dupe Collection: Completed 1.3.2022
# Real Collection: Completed 1.3.2022

# collection_to_update = db.sep_entries
# file_path='../downloads/articles/sep_2021_winter'
# file_type='*.html'
# sep_files = io.get_local_files(file_path,file_type)
# for sep_file in tqdm(sep_files):
#     sep_object = sep.parse_sep_file(sep_file)
#     update_string = sep.update_main_document_data(sep_object)
#     find = {"page_url": sep_object['page_url']}
#     collection_to_update.update_one(find, {'$set': update_string}, upsert=True)

In [13]:
#export copyright data into Pandas to retrieve author info  
##Winter 2021 completed 1.3.2022#

# collection_to_update = db.sep_entries_dupe

# copyright_entries = pd.DataFrame(list(collection_to_update.find(filter={'author': {'$exists': False}}, 
#                                                            projection=['page_url','copyright'])))

# # #strip HTML from copyright data, and re-write the copyright column data
# copyright_entries['copyright'] = copyright_entries.apply(lambda row:sep.get_author_string(row['copyright']), axis=1)
# #export CSV for cleaning
# copyright_entries.to_csv('coopyright_winter_2021_test.csv')

In [18]:
#update author data for SEP Winter 2021
#  completed 1.3.2022
# 

# collection_to_update = db.sep_entries

# #import cleaned data
# authors = pd.read_csv('coopyright_winter_2021.csv')

# for index,row in tqdm(authors.iterrows()):
#     page_url = row['page_url']
#     authors = row['copyright']
#     sep.add_author(page_url, authors, collection_to_update)